In [ ]:
import looker_sdk
from looker_sdk import api_settings

class MyApiSettings(api_settings.ApiSettings):
    def __init__(self, *args, **kw_args):
        super().__init__(*args, **kw_args)

    def read_config(self) -> api_settings.SettingsConfig:
        config = super().read_config()
        config["client_id"] = ""
        config["client_secret"] = ""
        config["base_url"] = "http://hack.looker.com:19999"

        return config

sdk = looker_sdk.init40(config_settings=MyApiSettings())

In [ ]:
import json 
import csv

def getRows(tableName):
    artifacts = sdk.search_artifacts(key=tableName+'%', namespace='Hackathon')
    rows = []
    for a in artifacts:
        row = json.loads(a.value)
        row['key'] = a.key
        rows.append(row)
    return rows

def toCSV(rows, fileName):
    if rows.count == 0:
        raise Exception('No artifacts in table!')
    with open(fileName, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        header = [x for x in rows[0].keys()]
        csv_rows = [header]
        for row in rows:
            csv_row = []
            for h in header:
                csv_row.append(row[h])
            csv_rows.append(csv_row)
        
        writer.writerows(csv_rows)

In [ ]:
def getJudgingResults(hackathonId, lookerGroupId=None):
    judgeUserKeys = None
    if lookerGroupId is not None:
        lookerUsers = sdk.all_group_users(lookerGroupId)
        print('Judge subset:')
        print([u.first_name + ' ' + u.last_name for u in lookerUsers])
        lookerUserIds = [u.id for u in lookerUsers]
        userRows = filter(lambda u: u['looker_id'] in lookerUserIds, getRows('User'))
        judgeUserKeys = [u['key'] for u in userRows]

    projects = [p for p in getRows('Project') if p['_hackathon_id'] == hackathonId]
    judgings = getRows('Judging')
    for p in projects:
        scores = []
        projectJudgings = filter(lambda j: j['project_id'] == p['key'], judgings)
        if judgeUserKeys is not None:
            projectJudgings = filter(lambda j: j['user_id'] in judgeUserKeys, projectJudgings)
        
        for j in projectJudgings:
            scores.append(int(j['score']))

        p['score'] = 0 if len(scores) == 0 else sum(scores)/len(scores)
    
    scoreRows = [{'score': p['score'], 'title': p['title'], 'key': p['key']} for p in projects]
    toCSV(scoreRows, 'Proj-scores.csv')

getJudgingResults('Hackathon:join_2021', '15')

In [22]:
toCSV(getRows('Hackathon'), 'Hackathon.csv')
toCSV(getRows('Project'), 'Project.csv')
toCSV(getRows('Judging'), 'Judging.csv')
toCSV(getRows('User'), 'User.csv')
toCSV(getRows('Technology'), 'Technology.csv')
toCSV(getRows('TeamMember'), 'TeamMember.csv')
toCSV(getRows('Registration'), 'Registration.csv')


In [ ]:
# Migrate all data 
# Download each google sheet as csv
# Expects directory/naming to be: csv/hackathons.csv
import json
import csv

def toValues(path):
    with open(path, newline='') as csvfile:
        reader = csv.reader(x.replace('\0', '') for x in csvfile)
        header = next(reader)
        values = []
        for row in reader:
            value = {}
            for index, v in enumerate(row):
                value[header[index]] = v
            values.append(value)
        return values
    
def toArtifact(key, value):
    artifact = {}
    artifact['key'] = key
    artifact['content_type'] = 'application/json'
    artifact['value'] = json.dumps(value)
    return artifact
    
artifacts = []

for v in toValues('csv/hackathons.csv'):
    key = 'Hackathon:' + v['_id']
    del v['_id'] 
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/judgings.csv'):
    key = 'Judging:' + v['_id']
    del v['_id'] 
    v['user_id'] = 'User:' + v['user_id']
    v['project_id'] = 'Project:' + v['project_id']
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/projects.csv'):
    key = 'Project:' + v['_id']
    del v['_id']
    v['_user_id'] = 'User:' + v['_user_id']
    v['_hackathon_id'] = 'Hackathon:' + v['_hackathon_id']
    v['technologies'] = ['Technology:' + x for x in v['technologies'].split(',')]
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/registrations.csv'):
    key = 'Registration:' + v['_id']
    del v['_id'] 
    v['_user_id'] = 'User:' + v['_user_id']
    v['hackathon_id'] = 'Hackathon:' + v['hackathon_id']
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/team_members.csv'):
    key = 'TeamMember:' + v['_id']
    del v['_id'] 
    v['user_id'] = 'User:' + v['user_id']
    v['project_id'] = 'Project:' + v['project_id']
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/technologies.csv'):
    key = 'Technology:' + v['_id']
    del v['_id'] 
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/users.csv'):
    key = 'User:' + v['_id']
    v['looker_id'] = '' + v['_id']
    del v['_id'] 
    artifacts.append(toArtifact(key, v))

sdk.update_artifacts('hackathon', artifacts)


In [ ]:
# TO PURGE ALL TABLES. BE CAREFUL
sdk.purge_artifacts('hackathon')
